# Task 1 - Problem Formulation

Task 1 requires defining each machine learning problem using the **T-P-E framework: Task, Performance, Experience**

## 1.1 Power Forecasting

The goal is to predict solar power output at inverter level.
### Task (T)
* **Target Variable:** DC_POWER or AC_POWER
* **ML Problem Type:** Suoervised Learning - Regression
* **Prediction Horizon:** Predict instataneous power output at the same timestamp using available features

### Performance (P)
* **Metric:** MAE, RMSE and $R^2$
* **Justification:**
    * **MAE:** gives interpretable average error
    * **RMSE:** penalises large deviations which is important due to cloud variability
    * **$R^2$:** quantifies variance explained by the model

### Experience (E) - (Refer to general data description below)

## 1.2 Operating Condition Classification

... (Repeat T, P, E)

## 1.3 Temporal Forecasting

... (Repeat T, P, E)

## 1.4 General Data Experience (E Summary)
* **Available Features:** ...
* **Temporal Coverage:** ...
* **Quality Issues:** ...
* **Splitting Strategy:** ... (e.g., Chronological 70/15/15 split)

test